In [2]:
import pandas as pd
import os
from model import NLUModel
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import random
import sys
import math

DATA_DIRECTORY = '../data/'
ROC_TRAIN_SET = 'train_stories.csv'
ROC_VAL_SET = 'cloze_test_val__spring2016 - cloze_test_ALL_val.csv'
ROC_TEST_SET = 'test_for_report-stories-labels.csv'

TRAIN_SKIP_THOUGHTS_EMBEDDINGS = '/cluster/project/infk/courses/machine_perception_19/Sasglentamekaiedo/skip-thoughts-embbedings.npy'
VAL_SKIP_THOUGHTS_EMBEDDINGS = '/cluster/project/infk/courses/machine_perception_19/Sasglentamekaiedo/skip-thoughts-embbedings_validation.npy'
TEST_SKIP_THOUGHTS_EMBEDDINGS = '/cluster/project/infk/courses/machine_perception_19/Sasglentamekaiedo/skip-thoughts-embbedings_test.npy'

LOG_PATH = '../log_path'

# parameter
num_labels = 2

# create dataset
def create_dataset_from_embeddings(embeddings, df):
    v_embeddings = list()
    v_classes = list()
    correct_answers = df['AnswerRightEnding'].values

    for i, story_embedding in enumerate(embeddings):
        v_embeddings.append(np.append(story_embedding[:4], [story_embedding[4]], axis=0))
        v_embeddings.append(np.append(story_embedding[:4], [story_embedding[5]], axis=0))

        if correct_answers[i] == 1:
            v_classes.append(0)
            v_classes.append(1)
        else:
            v_classes.append(1)
            v_classes.append(0)

    return np.array(v_embeddings), np.array(v_classes)


def get_final_predictions(probabilities):
    # predictions based on probabilities!
    my_predictions = []

    probabilities_exp = np.exp(probabilities)

    i = 0
    while i < len(probabilities):
        p_first = probabilities_exp[i]
        p_second = probabilities_exp[i + 1]

        p1 = p_first[0] + p_second[1]
        p2 = p_first[1] + p_second[0]

        if p1 > p2:
            my_predictions.append(0)
        else:
            my_predictions.append(1)
        i += 2

    return np.array(my_predictions)


class Encoder(tf.keras.Model):
    def __init__(self, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units

        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self, batch_size):
        return tf.zeros((batch_size, self.enc_units))


class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()

        self.W1 = tf.keras.layers.Dense(units, name='Dense_1')
        self.W2 = tf.keras.layers.Dense(units, name='Dense_2')
        self.V = tf.keras.layers.Dense(1, name='Dense_3')

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class DenseLayerWithSoftmax(tf.keras.Model):
    def __init__(self, layers, num_classes, dropout_keep_proba=0.9, activation=tf.nn.relu):
        super(DenseLayerWithSoftmax, self).__init__()

        self.dense_layers = []
        self.dropout_keep_proba = dropout_keep_proba
        self.num_classes = num_classes

        for i, layer_size in enumerate(layers):
            self.dense_layers.append(
                tf.keras.layers.Dense(layer_size, name='DenseLayer_' + str(i), use_bias=True, activation=tf.nn.relu))

        self.final_layer = tf.keras.layers.Dense(self.num_classes, name='DenseLayer_final', use_bias=True)

    def call(self, input_, input_labels):

        logits = input_

        for layer in self.dense_layers:
            logits = layer(logits)
            logits = tf.nn.dropout(logits, keep_prob=self.dropout_keep_proba)

        logits = self.final_layer(logits)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(input_labels, depth=self.num_classes, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))

        per_example_loss = - tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)

        return predicted_labels, loss, log_probs


class FCSkip(tf.keras.Model):
    def __init__(self, units,num_classes=2, fc_layers=[], dropout_keep_prob=0.9, activation=tf.nn.relu):
        super(FCSkip, self).__init__()

        self.units = units

        self.encoder = Encoder(self.units)
        self.feed_forward = DenseLayerWithSoftmax(fc_layers, num_classes,
                                                 dropout_keep_proba=dropout_keep_prob, activation=activation)

    def call(self, input_embeddings, input_labels):

        # sample input
        sample_hidden = self.encoder.initialize_hidden_state(tf.shape(input_embeddings)[0])
        sample_output, sample_hidden = self.encoder(input_embeddings[:, :4, :], sample_hidden)

        concatenated_input = tf.concat([sample_output[:, -1, :], input_embeddings[:, 4, :]], axis=1)

        return self.feed_forward(concatenated_input, input_labels)


class LSSkip(tf.keras.Model):
    def __init__(self, units,num_classes=2, fc_layers=[], dropout_keep_prob=0.9, activation=tf.nn.relu):
        super(LSSkip, self).__init__()

        self.units = units

        self.feed_forward = DenseLayerWithSoftmax(fc_layers, num_classes,
                                                 dropout_keep_proba=dropout_keep_prob, activation=activation)

    def call(self, input_embeddings, input_labels):
        concatenated_input = tf.concat([input_embeddings[:, 3, :], input_embeddings[:, 4, :]], axis=1)
#         concatenated_input = input_embeddings[:, 3, :] + input_embeddings[:, 4, :]

        return self.feed_forward(concatenated_input, input_labels)


class SimpleAndEffectiveApproach(NLUModel):
    train_embeddings=None
    train_classes=None
    test_embeddings=None
    test_classes=None
    batch_size=None

    def __init__(self, units, fc_layers=None, num_classes=2, train_on_validation=False, train_split=0.8, mode='FC-skip', verbose=False, negative_sampling=3, learning_rate=1e-3):
        assert mode in ['FC-skip', 'LS-skip'], "mode specified not supported"

        super(SimpleAndEffectiveApproach, self).__init__('SimpleAndEffectiveApproach')
        self.train_on_validation = train_on_validation
        self.train_split = train_split
        self.verbose = verbose
        self.negative_sampling = negative_sampling
        self.mode = mode
        self.fc_layers = fc_layers
        self.units = units
        self.learning_rate = learning_rate

    def _create_graph(self):
        train_x, train_y, test_x, test_y = self.__prepare_embeddings()

        if self.mode == 'FC-skip':
            if self.fc_layers is None:
                print('Using default values for feed forward networ [256, 64]')
                self.fc_layers = [256, 64]
            fc_skip = FCSkip(self.units, num_classes=2, fc_layers=self.fc_layers,
                                        dropout_keep_prob=0.9, activation=tf.nn.relu)

            self.predicted_labels_train, self.loss_train, self.log_probs_train = fc_skip(train_x, train_y)
            self.predicted_labels_test, self.loss_test, self.log_probs_test = fc_skip(test_x, test_y)

        elif self.mode == 'LS-skip':
            if self.fc_layers is None:
                print('Using default values for feed forward networ [2400, 1200, 600]')
                self.fc_layers = [2400, 1200, 600]

            ls_skip = LSSkip(self.units, num_classes=2, fc_layers=self.fc_layers, dropout_keep_prob=0.9, activation=tf.nn.relu)

            self.predicted_labels_train, self.loss_train, self.log_probs_train = ls_skip(train_x, train_y)
            self.predicted_labels_test, self.loss_test, self.log_probs_test = ls_skip(test_x, test_y)

    def __prepare_embeddings(self):
        if not self.train_on_validation:
            data_train = pd.read_csv(os.path.join(DATA_DIRECTORY, ROC_VAL_SET), header='infer')
            # has a shape (88161, 5, 4800)
            train_embeddings = np.load(TRAIN_SKIP_THOUGHTS_EMBEDDINGS)

        data_val = pd.read_csv(os.path.join(DATA_DIRECTORY, VAL_SET), header='infer')
        # has a shape (1871, 6, 4800)
        validation_embeddings = np.load(VAL_SKIP_THOUGHTS_EMBEDDINGS)

        # create set for validation dataset
        val_embeddings, val_classes = create_dataset_from_embeddings(validation_embeddings, data_val)

        if self.train_on_validation:
            np.random.seed(42)

            msk = np.random.rand(len(val_embeddings) // 2) < self.train_split
            new_mask = np.array([[value, value] for value in msk]).reshape(-1)

            self.train_embeddings, self.train_classes = shuffle(val_embeddings[new_mask], val_classes[new_mask])

            self.test_embeddings, self.test_classes = val_embeddings[~new_mask], val_classes[~new_mask]
        else:
            # noinspection PyUnboundLocalVariable
            self.train_embeddings, self.train_classes = shuffle(train_embeddings), np.zeros(len(train_embeddings))

            self.test_embeddings, self.test_classes = val_embeddings, val_classes

        self.num_samples_training = len(self.train_embeddings)
        self.num_samples_validation = len(self.test_embeddings)

        if self.train_on_validation:
            print('Loading without negative sampling')
            dataset_train = tf.data.Dataset.from_tensor_slices((self.train_embeddings, self.train_classes))
            dataset_train = dataset_train.shuffle(buffer_size=len(self.train_embeddings))

            dataset_train = dataset_train.repeat()
            dataset_train = dataset_train.batch(self.batch_size)

            iterator_train = dataset_train.make_one_shot_iterator()
            train_x, train_y = iterator_train.get_next()

            dataset_test = tf.data.Dataset.from_tensor_slices((self.test_embeddings, self.test_classes))

            dataset_test = dataset_test.repeat()
            dataset_test = dataset_test.batch(self.batch_size)

            iterator_test = dataset_test.make_one_shot_iterator()
            test_x, test_y = iterator_test.get_next()
        else:
            train_x, train_y = self.create_dataset(True, self.batch_size, self.negative_sampling)

            test_x, test_y = self.create_dataset(False, self.batch_size, 0)

        return train_x, train_y, test_x, test_y

    @staticmethod
    def sample_negatives(embeddings, embeddings_batch, classes_batch, negative_sampling):
        new_classes = []
        new_embeddings_batch = []
        for i, embedding in enumerate(embeddings_batch):

            new_embeddings_batch.append(embedding)
            new_classes.append(classes_batch[i])

            for _ in range(negative_sampling):
                new_embeddings_batch.append(
                    np.concatenate((embedding[:4], [random.choice(embeddings[:, 4, :])]), axis=0))
                # negative class always
                new_classes.append(1)
        return np.array(new_embeddings_batch, dtype=np.float32), np.array(new_classes, dtype=np.int32)

    def generator(self, train, batch_size=64, negative_sampling=0):
        """
        negative_sampling: For each positive sample these many negatives
        """
        if train:
            embeddings = self.train_embeddings
            classes = self.train_classes
        else:
            embeddings = self.test_embeddings
            classes = self.test_classes

        if train and negative_sampling > 0:
            batch_size /= (negative_sampling + 1)
            if batch_size != int(batch_size):
                raise Exception('Batch size should be an integer. Please change negative sampling rate')

            batch_size = int(batch_size)

        # repeat
        while True:
            if train:
                embeddings, classes = shuffle(embeddings, classes)

            length = len(embeddings)
            for ndx in range(0, length, batch_size):
                embeddings_batch = embeddings[ndx: min(ndx + batch_size, length)]
                classes_batch = classes[ndx: min(ndx + batch_size, length)]

                if negative_sampling <= 0:
                    yield embeddings_batch, classes_batch
                else:
                    yield self.sample_negatives(embeddings, embeddings_batch, classes_batch, negative_sampling)

    def create_dataset(self, train, batch_size, negative_sampling):
        dataset = tf.data.Dataset.from_generator(self.generator, (tf.float32, tf.int32),
                                                 output_shapes=(
                                                 tf.TensorShape([None, 5, 4800]), tf.TensorShape([None])),
                                                 args=([train, batch_size, negative_sampling]))

        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()

    def _predict_wth_session(self, sess):
        predictions = []
        for _ in range(math.ceil(self.num_samples_validation // self.batch_size)):
            
            predictions_batch = sess.run([self.log_probs_test])
            predictions.append(predictions_batch)

        return np.concatenate(predictions, axis=0).reshape(-1, 2)
    
    def predict(self, X):
        """
        X should always be the validation set?
        """

        with tf.Session() as sess:

            saver = tf.train.Saver()
            saver.restore(sess, tf.train.latest_checkpoint(LOG_PATH))

            return self._predict_wth_session(sess)

    def fit(self, X, y, epochs=10, batch_size=64):
        """
        X, y are unused in this case
        """

        self.batch_size = batch_size
        self._create_graph()
        
        update_learning_rate = 20 # empirical
        if self.train_on_validation:
            update_lr_every = int((math.ceil(self.num_samples_training / batch_size) * epochs) / update_learning_rate)
        else:
            update_lr_every = int(
                (math.ceil(self.num_samples_training / (batch_size / (self.negative_sampling + 1))) * epochs) / update_learning_rate)

        global_step = tf.Variable(0, trainable=False, name="global_step")
        if self.verbose:
            print('Updating the learning rate every:', update_lr_every, 'steps.')

        # learning rate 1e-3 for most models
        learning_rate = tf.train.exponential_decay(
            self.learning_rate,  # Base learning rate.
            global_step,  # Current index into the dataset.
            update_lr_every,  # Decay step.
            0.96,  # Decay rate.
            staircase=True)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        gradients, variables = zip(*optimizer.compute_gradients(self.loss_train))
        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        train_op = optimizer.apply_gradients(zip(gradients, variables), global_step=global_step)

        with tf.Session() as sess:
            tf.global_variables_initializer().run()

            # define model saver
            saver = tf.train.Saver(tf.global_variables())

            #     with trange(math.ceil(NUM_SAMPLES_TRAINING / (batch_size / (negative_sampling + 1))) * NUM_EPOCHS) as t:
            if self.train_on_validation:
                number_of_steps = math.ceil(self.num_samples_training / batch_size)
            else:
                number_of_steps = math.ceil(self.num_samples_training/ (batch_size / (self.negative_sampling + 1)))

            last_epoch = 0
            for i in range(number_of_steps * epochs):
                # display training status
                epoch_cur = i // number_of_steps

                _, _, lt = sess.run([train_op, global_step, self.loss_train])

                if epoch_cur > last_epoch and self.verbose:
                    # print score on validation set
                    last_epoch = epoch_cur
                    log_predictions_test = self._predict_wth_session(sess)

                    print('At epoch', last_epoch, 'score on validation set',
                          accuracy_score(self.test_classes[::2], get_final_predictions(log_predictions_test)))

            saver.save(sess,os.path.join(LOG_PATH,"model"),
                       global_step=int(epochs * self.num_samples_training / batch_size))

In [3]:
simpleAndEffectiveApproach = SimpleAndEffectiveApproach(4800, train_on_validation=True, train_split=0.8, mode='LS-skip', verbose=True)

In [4]:
simpleAndEffectiveApproach.fit(None, None, batch_size=32, epochs=20)

Loading without negative sampling
Using default values for feed forward networ [2400, 1200, 600]
Updating the learning rate every: 93 steps.
At epoch 1 score on validation set 0.6901041666666666
At epoch 2 score on validation set 0.7265625
At epoch 3 score on validation set 0.7552083333333334
At epoch 4 score on validation set 0.7369791666666666
At epoch 5 score on validation set 0.7526041666666666
At epoch 6 score on validation set 0.75
At epoch 7 score on validation set 0.7369791666666666
At epoch 8 score on validation set 0.765625
At epoch 9 score on validation set 0.765625
At epoch 10 score on validation set 0.7786458333333334
At epoch 11 score on validation set 0.765625
At epoch 12 score on validation set 0.78125
At epoch 13 score on validation set 0.7526041666666666
At epoch 14 score on validation set 0.7604166666666666
At epoch 15 score on validation set 0.78125
At epoch 16 score on validation set 0.8020833333333334
At epoch 17 score on validation set 0.7604166666666666
At epoch

In [15]:
batch_size = 64
epochs = 10

if simpleAndEffectiveApproach.train_on_validation:
    update_lr_every = int((math.ceil(simpleAndEffectiveApproach.num_samples_training / batch_size) * epochs) / 20)
else:
    update_lr_every = int(
        (math.ceil(simpleAndEffectiveApproach.num_samples_training / (batch_size / (simpleAndEffectiveApproach.negative_sampling + 1))) * epochs) / 20)

global_step = tf.Variable(0, trainable=False, name="global_step")

# learning rate 1e-3 for most models
learning_rate = tf.train.exponential_decay(
    1e-7,  # Base learning rate.
    global_step,  # Current index into the dataset.
    update_lr_every,  # Decay step.
    0.96,  # Decay rate.
    staircase=True)

optimizer = tf.train.AdamOptimizer(learning_rate)
gradients, variables = zip(*optimizer.compute_gradients(simpleAndEffectiveApproach.loss_train))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
train_op = optimizer.apply_gradients(zip(gradients, variables), global_step=global_step)

In [16]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    # define model saver
    saver = tf.train.Saver(tf.global_variables())

    #     with trange(math.ceil(NUM_SAMPLES_TRAINING / (batch_size / (negative_sampling + 1))) * NUM_EPOCHS) as t:
    if simpleAndEffectiveApproach.train_on_validation:
        number_of_steps = math.ceil(simpleAndEffectiveApproach.num_samples_training / batch_size)
    else:
        number_of_steps = math.ceil(simpleAndEffectiveApproach.num_samples_training/ (batch_size / (simpleAndEffectiveApproach.negative_sampling + 1)))

    last_epoch = 0
    
    for i in range(number_of_steps * epochs):
        # display training status
        epoch_cur = i // number_of_steps

        _, _, lt = sess.run([train_op, global_step, simpleAndEffectiveApproach.loss_train])

        if epoch_cur > last_epoch and simpleAndEffectiveApproach.verbose:
            # print score on validation set
            last_epoch = epoch_cur
            predictions_test = simpleAndEffectiveApproach._predict_wth_session(sess)

            print('At epoch', last_epoch, 'score on validation set',
                  accuracy_score(simpleAndEffectiveApproach.test_classes[::2], get_final_predictions(predictions_test)))

    saver.save(sess,os.path.join(LOG_PATH,"model"),
               global_step=int(epochs * simpleAndEffectiveApproach.num_samples_training / batch_size))


47
0
[array([[-0.7155281 , -0.67125624],
       [-0.7154268 , -0.6713531 ],
       [-0.72295684, -0.66420037],
       [-0.723681  , -0.66351813],
       [-0.7177918 , -0.66909534],
       [-0.7318429 , -0.65589327],
       [-0.72427875, -0.6629556 ],
       [-0.7036398 , -0.6827635 ],
       [-0.71955085, -0.6674227 ],
       [-0.73473644, -0.65321875],
       [-0.727928  , -0.6595355 ],
       [-0.7215555 , -0.66552365],
       [-0.7355051 , -0.6525109 ],
       [-0.7167204 , -0.67011696],
       [-0.7223722 , -0.664752  ],
       [-0.7271663 , -0.6602473 ],
       [-0.7298918 , -0.657705  ],
       [-0.72450244, -0.6627454 ],
       [-0.7309692 , -0.65670365],
       [-0.7223205 , -0.6648009 ],
       [-0.7383365 , -0.64991194],
       [-0.72812617, -0.65935034],
       [-0.7313218 , -0.6563765 ],
       [-0.72229946, -0.6648207 ],
       [-0.73826665, -0.64997584],
       [-0.7246397 , -0.66261625],
       [-0.71469486, -0.67205405],
       [-0.726898  , -0.66049844],
       [-0.716

[array([[-0.7262498 , -0.66110533],
       [-0.69813687, -0.68818235],
       [-0.7191555 , -0.6677981 ],
       [-0.7058365 , -0.68061674],
       [-0.71382594, -0.6728873 ],
       [-0.7253119 , -0.66198504],
       [-0.73476243, -0.65319484],
       [-0.7108325 , -0.6757692 ],
       [-0.7249161 , -0.6623565 ],
       [-0.72626966, -0.6610867 ],
       [-0.74044126, -0.64798915],
       [-0.7201604 , -0.6668445 ],
       [-0.7334502 , -0.65440583],
       [-0.72415686, -0.66307014],
       [-0.7349795 , -0.6529947 ],
       [-0.73982894, -0.6485479 ],
       [-0.7325264 , -0.6552601 ],
       [-0.73011804, -0.65749454],
       [-0.73592365, -0.65212566],
       [-0.7214853 , -0.66559005],
       [-0.71853644, -0.6683865 ],
       [-0.74213445, -0.646448  ],
       [-0.7287694 , -0.6587504 ],
       [-0.7071255 , -0.6793615 ],
       [-0.70830595, -0.67821485],
       [-0.722603  , -0.6645342 ],
       [-0.7418686 , -0.6466896 ],
       [-0.7224703 , -0.66465944],
       [-0.7383303 

[array([[-0.72785217, -0.6596064 ],
       [-0.71100974, -0.67559814],
       [-0.73966825, -0.64869446],
       [-0.7232456 , -0.66392833],
       [-0.7103398 , -0.67624515],
       [-0.73706293, -0.65107906],
       [-0.7272371 , -0.66018116],
       [-0.72391427, -0.66329855],
       [-0.72395915, -0.66325635],
       [-0.7436213 , -0.6450987 ],
       [-0.72636753, -0.6609949 ],
       [-0.72489077, -0.6623802 ],
       [-0.731324  , -0.6563743 ],
       [-0.7290575 , -0.65848184],
       [-0.7134764 , -0.673223  ],
       [-0.72318226, -0.66398793],
       [-0.7266078 , -0.66077006],
       [-0.72582084, -0.66150737],
       [-0.72004044, -0.6669582 ],
       [-0.7312245 , -0.65646666],
       [-0.726743  , -0.6606434 ],
       [-0.72943807, -0.65812725],
       [-0.7296413 , -0.6579381 ],
       [-0.73438096, -0.6535465 ],
       [-0.73777634, -0.6504249 ],
       [-0.7170806 , -0.6697731 ],
       [-0.7294016 , -0.65816134],
       [-0.7305789 , -0.6570661 ],
       [-0.72680205

[array([[-0.71526206, -0.6715109 ],
       [-0.722523  , -0.66460973],
       [-0.72121125, -0.66584915],
       [-0.72367847, -0.6635205 ],
       [-0.72928005, -0.6582744 ],
       [-0.7325491 , -0.65523905],
       [-0.72442394, -0.662819  ],
       [-0.7226942 , -0.6644483 ],
       [-0.7279285 , -0.659535  ],
       [-0.72859657, -0.6589115 ],
       [-0.7184034 , -0.6685131 ],
       [-0.7276488 , -0.65979624],
       [-0.73036   , -0.6572696 ],
       [-0.71854746, -0.66837615],
       [-0.7286464 , -0.65886515],
       [-0.7276365 , -0.65980786],
       [-0.7207648 , -0.66627175],
       [-0.72026706, -0.66674346],
       [-0.73925716, -0.64906996],
       [-0.72216237, -0.6649502 ],
       [-0.7108683 , -0.6757346 ],
       [-0.71835136, -0.6685626 ],
       [-0.7266766 , -0.66070557],
       [-0.7364063 , -0.65168214],
       [-0.7398288 , -0.6485477 ],
       [-0.7229058 , -0.66424865],
       [-0.72667676, -0.6607054 ],
       [-0.72911495, -0.6584283 ],
       [-0.7243796 

[array([[-0.71917677, -0.6677781 ],
       [-0.7182568 , -0.66865265],
       [-0.7050295 , -0.68140435],
       [-0.7186382 , -0.6682898 ],
       [-0.7383473 , -0.649902  ],
       [-0.7168502 , -0.66999286],
       [-0.72188157, -0.6652155 ],
       [-0.72874653, -0.65877163],
       [-0.72138846, -0.6656817 ],
       [-0.7378339 , -0.6503722 ],
       [-0.71969664, -0.6672844 ],
       [-0.7297341 , -0.6578518 ],
       [-0.7217257 , -0.6653627 ],
       [-0.7190792 , -0.6678706 ],
       [-0.7254645 , -0.6618416 ],
       [-0.7233775 , -0.6638039 ],
       [-0.72045845, -0.66656196],
       [-0.72613686, -0.6612112 ],
       [-0.7351608 , -0.6528276 ],
       [-0.71835256, -0.66856164],
       [-0.7323763 , -0.65539914],
       [-0.7272207 , -0.6601964 ],
       [-0.7092983 , -0.6772528 ],
       [-0.7330683 , -0.6547589 ],
       [-0.7240977 , -0.66312593],
       [-0.7325953 , -0.6551962 ],
       [-0.7538288 , -0.6359381 ],
       [-0.7317479 , -0.6559813 ],
       [-0.73866415

[array([[-0.7265817 , -0.6607945 ],
       [-0.71249795, -0.6741637 ],
       [-0.733192  , -0.65464425],
       [-0.71979487, -0.6671912 ],
       [-0.72612625, -0.661221  ],
       [-0.72873354, -0.6587838 ],
       [-0.7147999 , -0.6719533 ],
       [-0.71267045, -0.6739977 ],
       [-0.72696936, -0.6604317 ],
       [-0.7335847 , -0.65428144],
       [-0.7158321 , -0.6709655 ],
       [-0.7344373 , -0.65349466],
       [-0.72650135, -0.66086966],
       [-0.7156729 , -0.6711177 ],
       [-0.7272697 , -0.66015065],
       [-0.70619434, -0.68026805],
       [-0.7111236 , -0.6754882 ],
       [-0.7284412 , -0.65905654],
       [-0.7286856 , -0.6588285 ],
       [-0.70922095, -0.67732763],
       [-0.73639804, -0.6516896 ],
       [-0.70678145, -0.6796964 ],
       [-0.7198546 , -0.6671346 ],
       [-0.7265725 , -0.66080296],
       [-0.71511966, -0.67164713],
       [-0.72938   , -0.6581814 ],
       [-0.72668   , -0.6607024 ],
       [-0.71614397, -0.6706673 ],
       [-0.7219883 

300
[array([[-0.720919  , -0.6661259 ],
       [-0.7037629 , -0.68264306],
       [-0.73349977, -0.6543599 ],
       [-0.71705526, -0.6697974 ],
       [-0.7204926 , -0.6665296 ],
       [-0.727437  , -0.6599943 ],
       [-0.72206336, -0.66504365],
       [-0.7032043 , -0.68319017],
       [-0.71931475, -0.6676469 ],
       [-0.7288401 , -0.6586844 ],
       [-0.7289306 , -0.6586001 ],
       [-0.74593925, -0.6430028 ],
       [-0.7371899 , -0.6509627 ],
       [-0.7215155 , -0.6655615 ],
       [-0.7205381 , -0.66648656],
       [-0.72376436, -0.66343963],
       [-0.71779007, -0.6690971 ],
       [-0.71367383, -0.6730334 ],
       [-0.72461593, -0.6626386 ],
       [-0.7255602 , -0.66175187],
       [-0.7351781 , -0.6528119 ],
       [-0.719553  , -0.6674207 ],
       [-0.7228993 , -0.66425467],
       [-0.72717375, -0.66024035],
       [-0.7158618 , -0.670937  ],
       [-0.7284371 , -0.6590603 ],
       [-0.732789  , -0.655017  ],
       [-0.7211574 , -0.6659001 ],
       [-0.7131

[array([[-0.71839875, -0.66851753],
       [-0.7088952 , -0.67764324],
       [-0.7144373 , -0.67230093],
       [-0.72281134, -0.66433775],
       [-0.7167142 , -0.6701228 ],
       [-0.74017954, -0.6482278 ],
       [-0.72058135, -0.6664456 ],
       [-0.71068716, -0.6759095 ],
       [-0.70647484, -0.67999476],
       [-0.7286576 , -0.65885466],
       [-0.7366873 , -0.65142393],
       [-0.7319796 , -0.6557665 ],
       [-0.7279847 , -0.65948254],
       [-0.71073985, -0.6758587 ],
       [-0.72654605, -0.6608278 ],
       [-0.7095184 , -0.67703956],
       [-0.7120166 , -0.6746273 ],
       [-0.70883864, -0.677698  ],
       [-0.7203242 , -0.6666891 ],
       [-0.7149955 , -0.6717659 ],
       [-0.7254144 , -0.6618887 ],
       [-0.7075741 , -0.67892545],
       [-0.7283896 , -0.6591046 ],
       [-0.73661095, -0.651494  ],
       [-0.7094893 , -0.6770678 ],
       [-0.722008  , -0.665096  ],
       [-0.7423902 , -0.64621574],
       [-0.70769686, -0.6788061 ],
       [-0.7307718 

400
[array([[-0.7180864 , -0.6688147 ],
       [-0.72428656, -0.66294825],
       [-0.7376971 , -0.65049756],
       [-0.7166314 , -0.6702018 ],
       [-0.7299468 , -0.6576538 ],
       [-0.7212189 , -0.66584194],
       [-0.7275136 , -0.65992266],
       [-0.7024894 , -0.6838915 ],
       [-0.7164388 , -0.67038584],
       [-0.7131688 , -0.6735185 ],
       [-0.7339928 , -0.6539046 ],
       [-0.7317751 , -0.65595615],
       [-0.73983926, -0.64853823],
       [-0.71704715, -0.6698051 ],
       [-0.716263  , -0.6705536 ],
       [-0.724521  , -0.66272783],
       [-0.7289688 , -0.6585645 ],
       [-0.71794593, -0.6689486 ],
       [-0.74018925, -0.648219  ],
       [-0.72184944, -0.6652458 ],
       [-0.7252937 , -0.6620019 ],
       [-0.7163504 , -0.6704702 ],
       [-0.7254118 , -0.66189116],
       [-0.72643316, -0.66093355],
       [-0.72695553, -0.6604446 ],
       [-0.7287805 , -0.65874004],
       [-0.7260171 , -0.6613234 ],
       [-0.7229111 , -0.6642436 ],
       [-0.7363

InvalidArgumentError: Cannot serialize protocol buffer of type tensorflow.GraphDef as the serialized size (2156115397bytes) would be larger than the limit (2147483647 bytes)

In [ ]:
simpleAndEffectiveApproach.fit(None, None, epochs=5, batch_size=64)